# Running a regression on cleaned EOIR data

Do certain judges or cities report refugees more often? [This dataset is problematic](https://trac.syr.edu/immigration/reports/580/) and we're hardly experts, but let's take a look.

<p class="reading-options">
  <a class="btn" href="/reuters-asylum/using-regression-to-analyze-asylum-cases">
    <i class="fa fa-sm fa-book"></i>
    Read online
  </a>
  <a class="btn" href="/reuters-asylum/notebooks/Using regression to analyze asylum cases.ipynb">
    <i class="fa fa-sm fa-download"></i>
    Download notebook
  </a>
  <a class="btn" href="https://colab.research.google.com/github/littlecolumns/ds4j-notebooks/blob/master/reuters-asylum/notebooks/Using regression to analyze asylum cases.ipynb" target="_new">
    <i class="fa fa-sm fa-laptop"></i>
    Interactive version
  </a>
</p>

### Prep work: Downloading necessary files
Before we get started, we need to download all of the data we'll be using.
* **cases-filtered.csv:** filtered cases - filtered to a subset of comparable asylum cases
* **proceedings-filtered.csv:** filtered proceedings - filtered to a subset of comparable asylum cases
* **tblDecCode.csv:** decision codes - original decision codes csv file
* **tblLookupAlienNat.csv:** nationality lookup table - original nationality lookup table


In [ ]:
# Make data directory if it doesn't exist
!mkdir -p data
!wget -nc https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/reuters-asylum/data/cases-filtered.csv.zip -P data
!unzip -n -d data data/cases-filtered.csv.zip
!wget -nc https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/reuters-asylum/data/proceedings-filtered.csv.zip -P data
!unzip -n -d data data/proceedings-filtered.csv.zip
!wget -nc https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/reuters-asylum/data/tblDecCode.csv -P data
!wget -nc https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/reuters-asylum/data/tblLookupAlienNat.csv -P data

In [2]:
import pandas as pd

pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 500)
pd.set_option("display.float_format", "{:,.5f}".format)

> TODO: While this notebook is in theory complete, it needs to be revised to add a lot lot lot more words.

## Read in data

The data is split across two files - one for cases, and one for proceedings. We previously filtered them down to about 2 million cases from around 8-10 million, so we're working with the filtered version here.

> You'll also need to download the original dataset from [here](https://fileshare.eoir.justice.gov/FOIA-TRAC-Report.zip) for information on judges and nationalities.

In [3]:
cases = pd.read_csv("data/cases-filtered.csv")
cases.head()

,IDNCASE,NAT,CUSTODY,CASE_TYPE,UPDATE_SITE,DATE_OF_ENTRY
0,2048319,MX,N,RMV,MIA,1954-08-06 00:00:00.000
1,2048321,MX,N,RMV,HOU,NaN
2,2048337,MX,N,RMV,HOU,1992-01-01 00:00:00.000
3,2048340,CH,N,RMV,LOS,2000-04-01 00:00:00.000
4,2047883,TH,N,RMV,KAN,2002-05-10 00:00:00.000


In [4]:
proceedings = pd.read_csv("data/proceedings-filtered.csv")
proceedings.head()

,IDNCASE,OSC_DATE,IJ_CODE,DEC_TYPE,DEC_CODE,COMP_DATE,ABSENTIA,CASE_TYPE
0,3320158,2000-02-16 00:00:00.000,THS,O,X,2002-08-23 00:00:00.000,N,RMV
1,3320170,1998-12-23 00:00:00.000,HSD,O,T,1999-06-08 00:00:00.000,N,RMV
2,3320191,1998-04-30 00:00:00.000,PLM,NaN,NaN,2000-05-09 00:00:00.000,N,RMV
3,3320210,1997-06-03 00:00:00.000,BAN,O,V,2000-04-17 00:00:00.000,N,RMV
4,3320218,1997-09-10 00:00:00.000,EAL,O,V,1999-04-14 00:00:00.000,N,RMV


### Merge the datasets

We can now combine the two datasets based on the case IDs.

In [5]:
merged = cases.merge(proceedings, on='IDNCASE')
merged.shape

(3504464, 13)

In [6]:
merged.head(2)

,IDNCASE,NAT,CUSTODY,CASE_TYPE_x,UPDATE_SITE,DATE_OF_ENTRY,OSC_DATE,IJ_CODE,DEC_TYPE,DEC_CODE,COMP_DATE,ABSENTIA,CASE_TYPE_y
0,2048319,MX,N,RMV,MIA,1954-08-06 00:00:00.000,2001-02-13 00:00:00.000,PAM,O,T,2002-03-18 00:00:00.000,N,RMV
1,2048321,MX,N,RMV,HOU,NaN,1999-06-10 00:00:00.000,CMR,O,T,2000-02-02 00:00:00.000,N,RMV


## Remove missing data

We don't want anyone who is missing *any* of these fields. No date of entry? Out! No decision code? Out! No nationality? Out!

Let's take a quick look to see how _much_ missing data there is.

In [7]:
merged.isna().mean()

IDNCASE         0.00000
NAT             0.00093
CUSTODY         0.00000
CASE_TYPE_x     0.00000
UPDATE_SITE     0.00001
DATE_OF_ENTRY   0.26022
OSC_DATE        0.00294
IJ_CODE         0.01560
DEC_TYPE        0.51286
DEC_CODE        0.62581
COMP_DATE       0.26548
ABSENTIA        0.26861
CASE_TYPE_y     0.00000
dtype: float64

It looks like **around 63% don't have a decision**, which of course we can't analyze. Looks like we're going to be losing a lot lot lot of rows, though.

In [8]:
print("Before dropping missing data:", merged.shape)
merged = merged.dropna()
print("After dropping missing data:", merged.shape)

Before dropping missing data: (3504464, 13)
After dropping missing data: (1002980, 13)


That hurts, but maybe it's for the best!

## Analysis

### Decision codes

What are the different decision codes? Let's use the lookup table to see.

In [9]:
codes = pd.read_csv("data/tblDecCode.csv", sep='\t')
codes

,idnDecCode,strCode,strDescription,datCreatedOn,datModifiedOn,blnActive
0,1,A,LEGALLY ADMITTED,2003-08-10 11:22:47.617,NaN,1
1,2,C,CONDITIONAL GRANT,2003-08-10 11:22:47.617,NaN,1
2,3,D,DEPORTED,2003-08-10 11:22:47.617,NaN,1
3,4,E,EXCLUDED,2003-08-10 11:22:47.617,NaN,1
4,5,G,GRANTED,2003-08-10 11:22:47.617,2003-11-03 00:00:00.000,1
5,6,O,OTHER,2003-08-10 11:22:47.617,NaN,1
6,7,R,RELIEF/RESCINDED,2003-08-10 11:22:47.617,NaN,1
7,8,S,ALIEN MAINTAINS LEGAL STATUS,2003-08-10 11:22:47.617,NaN,1
8,9,T,CASE TERMINATED BY IJ,2003-08-10 11:22:47.617,NaN,1
9,10,V,VOLUNTARY DEPARTURE,2003-08-10 11:22:47.617,NaN,1


We're going to be interested in:

* `A - LEGALLY ADMITTED` as **successful**
* `C - CONDITIONAL GRANT` as **successful**
* `D - DEPORTED` as unsuccessful
* `G - GRANTED` as **successful**
* `R - RELIEF/RESCINDED` as **successful**
* `X - REMOVED` as unsuccessful

The others we'll remove, as we can't make a good judgment about what they really mean in the accepted/rejected spectrum.

Before we remove them, let's again see how common each code is.

In [10]:
merged.DEC_CODE.value_counts()

R    322356
X    257959
V    208949
T    192742
O      7625
D      4365
G      3426
Q      2913
I      1627
W       668
C       191
A       157
J         2
Name: DEC_CODE, dtype: int64

And now we'll filter for the decision codes we're interested in.

In [11]:
merged = merged[merged.DEC_CODE.isin(['A', 'C', 'D', 'G', 'R', 'X'])]
merged.head()

,IDNCASE,NAT,CUSTODY,CASE_TYPE_x,UPDATE_SITE,DATE_OF_ENTRY,OSC_DATE,IJ_CODE,DEC_TYPE,DEC_CODE,COMP_DATE,ABSENTIA,CASE_TYPE_y
3,2048340,CH,N,RMV,LOS,2000-04-01 00:00:00.000,2000-08-02 00:00:00.000,WJM,O,R,2000-12-04 00:00:00.000,N,RMV
5,2047890,NN,N,RMV,PHI,2049-08-31 00:00:00.000,1997-06-09 00:00:00.000,DVF,O,X,1998-04-23 00:00:00.000,N,RMV
8,2047893,MX,N,RMV,SFR,1954-06-25 00:00:00.000,2003-08-18 00:00:00.000,LLR,O,R,2004-11-29 00:00:00.000,N,RMV
11,2047908,CA,N,RMV,DET,1952-06-02 00:00:00.000,1997-06-10 00:00:00.000,RDN,O,R,2005-12-16 00:00:00.000,N,RMV
12,2047908,CA,N,RMV,DET,1952-06-02 00:00:00.000,1997-06-10 00:00:00.000,JFW,O,X,1998-12-11 00:00:00.000,N,RMV


In [12]:
merged.shape

(588454, 13)

**Down to around 600,000 cases!** Didn't we start with like **ten million** in the last notebook?

## Feature engineering

We're now going to create a feature as to **whether asylum was granted or not**. We'll count these:

* `A - LEGALLY ADMITTED`
* `C - CONDITIONAL GRANT`
* `G - GRANTED`
* `R - RELIEF/RESCINDED`

And the others - `D - DEPORTED` and `X - REMOVED` - we'll count as unsuccessful. We'll use the "turning true/false values into numbers" trick here.

In [13]:
merged['granted'] = merged.DEC_CODE.isin(['A', 'C', 'G', 'R']).astype(int)
merged.granted.value_counts()

1    326130
0    262324
Name: granted, dtype: int64

In [14]:
merged.DEC_TYPE.value_counts()

O    505468
W     79523
7      3460
6         2
C         1
Name: DEC_TYPE, dtype: int64

In [15]:
merged.head()

,IDNCASE,NAT,CUSTODY,CASE_TYPE_x,UPDATE_SITE,DATE_OF_ENTRY,OSC_DATE,IJ_CODE,DEC_TYPE,DEC_CODE,COMP_DATE,ABSENTIA,CASE_TYPE_y,granted
3,2048340,CH,N,RMV,LOS,2000-04-01 00:00:00.000,2000-08-02 00:00:00.000,WJM,O,R,2000-12-04 00:00:00.000,N,RMV,1
5,2047890,NN,N,RMV,PHI,2049-08-31 00:00:00.000,1997-06-09 00:00:00.000,DVF,O,X,1998-04-23 00:00:00.000,N,RMV,0
8,2047893,MX,N,RMV,SFR,1954-06-25 00:00:00.000,2003-08-18 00:00:00.000,LLR,O,R,2004-11-29 00:00:00.000,N,RMV,1
11,2047908,CA,N,RMV,DET,1952-06-02 00:00:00.000,1997-06-10 00:00:00.000,RDN,O,R,2005-12-16 00:00:00.000,N,RMV,1
12,2047908,CA,N,RMV,DET,1952-06-02 00:00:00.000,1997-06-10 00:00:00.000,JFW,O,X,1998-12-11 00:00:00.000,N,RMV,0


## More filters

We now want to filter our cases a bit more:

* Only judges who have had a certain number of cases
* Only sites that have had a certain number of cases
* Only nationalities that have had a certain number of cases.

If they haven't had many occurrences, we can't reasonably pass judgment on them. We'll start by making a copy of our dataset so we can filter it again/differently later.


### Filter judges

Let's only look at judges with 300 or more cases.

In [16]:
common_judges = list(merged.IJ_CODE.value_counts()[merged.IJ_CODE.value_counts() >= 300].index)
has_common_judge = merged.IJ_CODE.isin(common_judges)


And sites that have shown up 300 or more times

In [17]:
common_sites = list(merged.UPDATE_SITE.value_counts()[merged.UPDATE_SITE.value_counts() >= 300].index)
has_common_site = merged.UPDATE_SITE.isin(common_sites)
has_common_site.value_counts()

True     586894
False      1560
Name: UPDATE_SITE, dtype: int64

And nationalities that have shown up... 300 times?

In [18]:
common_nats = list(merged.NAT.value_counts()[merged.NAT.value_counts() >= 300].index)
has_common_nationality = merged.NAT.isin(common_nats)
has_common_nationality.value_counts()

True     579619
False      8835
Name: NAT, dtype: int64

In [19]:
filtered = merged[has_common_judge & has_common_site & has_common_nationality]
filtered.shape

(548433, 14)

### Inspecting our filtered results

In [20]:
filtered.IJ_CODE.value_counts().head()

ROS    5817
PLM    5423
TAB    5320
BLF    5286
LTB    5219
Name: IJ_CODE, dtype: int64

In [21]:
filtered.UPDATE_SITE.value_counts().head()

NYC    117866
MIA     81559
LOS     58525
SFR     43356
ORL     21764
Name: UPDATE_SITE, dtype: int64

In [22]:
filtered.NAT.value_counts().head()

CH    79263
MX    71713
ES    43123
GT    36133
HA    30280
Name: NAT, dtype: int64

## Perform our regression

Now that we've successfully filtered our data, we can run our regression. We'll control for **judges, cities, and nationalities**. We'll be using the statsmodels formula method, with `C()` for categorical variables. We'll use `ROS`, `NYC` as reference since they're the most popular judge and site. We'll use `CH` (China) as the reference for nationality because the results are more readable than if we used Mexico.

> No matter what you use as your reference the calculations are the same - it's just the point of comparison that is adjusted

In [23]:
%%time

import statsmodels.formula.api as smf

model = smf.logit("""
    granted ~ C(IJ_CODE, Treatment('ROS')) + C(UPDATE_SITE, Treatment('NYC')) + C(NAT, Treatment('CH'))
""", data=filtered)

CPU times: user 1min 13s, sys: 8.1 s, total: 1min 21s
Wall time: 1min 3s


It's a big dataset, so the actual fitting of the model takes a solid **7 minutes** on my computer.

In [24]:
%%time

result = model.fit(method='bfgs', maxiter=1000)
result.summary()

Optimization terminated successfully.
         Current function value: 0.553741
         Iterations: 442
         Function evaluations: 443
         Gradient evaluations: 443
CPU times: user 10min 40s, sys: 19.8 s, total: 11min
Wall time: 5min 48s


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                granted   No. Observations:               548433
Model:                          Logit   Df Residuals:                   547919
Method:                           MLE   Df Model:                          513
Date:                Sun, 19 Jan 2020   Pseudo R-squ.:                  0.1915
Time:                        13:36:03   Log-Likelihood:            -3.0369e+05
converged:                       True   LL-Null:                   -3.7561e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
===========================================================================================================
                                              coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------
Intercept                                   0.0616      0.052      1.192      0.233      -0.040       0.163
C(IJ_CODE, Treatment('ROS'))[T.AA]          0.2650      0.116      2.276      0.023       0.037       0.493
C(IJ_CODE, Treatment('ROS'))[T.AAK]         0.8781      0.100      8.816      0.000       0.683       1.073
C(IJ_CODE, Treatment('ROS'))[T.AAS]         1.0328      0.158      6.519      0.000       0.722       1.343
C(IJ_CODE, Treatment('ROS'))[T.AAT]        -0.3299      0.076     -4.322      0.000      -0.480      -0.180
C(IJ_CODE, Treatment('ROS'))[T.AAV]        -0.6595      0.073     -9.012      0.000      -0.803      -0.516
C(IJ_CODE, Treatment('ROS'))[T.ABM]         0.0499      0.138      0.362      0.718      -0.221       0.321
C(IJ_CODE, Treatment('ROS'))[T.ACH]         0.9506      0.133      7.143      0.000       0.690       1.211
C(IJ_CODE, Treatment('ROS'))[T.ADM]        -0.2655      0.144     -1.839      0.066      -0.548       0.017
C(IJ_CODE, Treatment('ROS'))[T.AED]         0.7910      0.095      8.336      0.000       0.605       0.977
C(IJ_CODE, Treatment('ROS'))[T.AEG]         1.0850      0.102     10.601      0.000       0.884       1.286
C(IJ_CODE, Treatment('ROS'))[T.AEM]        -1.0406      0.157     -6.617      0.000      -1.349      -0.732
C(IJ_CODE, Treatment('ROS'))[T.AJR]         1.2478      0.088     14.208      0.000       1.076       1.420
C(IJ_CODE, Treatment('ROS'))[T.ALP]        -0.2225      0.121     -1.833      0.067      -0.460       0.015
C(IJ_CODE, Treatment('ROS'))[T.AND]         0.4919      0.136      3.615      0.000       0.225       0.758
C(IJ_CODE, Treatment('ROS'))[T.AO]         -0.6558      0.067     -9.757      0.000      -0.788      -0.524
C(IJ_CODE, Treatment('ROS'))[T.ARD]        -0.0690      0.107     -0.646      0.518      -0.279       0.140
C(IJ_CODE, Treatment('ROS'))[T.ASE]         1.0163      0.070     14.525      0.000       0.879       1.153
C(IJ_CODE, Treatment('ROS'))[T.ASG]         0.6007      0.085      7.057      0.000       0.434       0.768
C(IJ_CODE, Treatment('ROS'))[T.ASL]         1.1510      0.082     13.968      0.000       0.990       1.313
C(IJ_CODE, Treatment('ROS'))[T.ASM]        -0.6735      0.096     -7.047      0.000      -0.861      -0.486
C(IJ_CODE, Treatment('ROS'))[T.ATG]         0.1473      0.081      1.824      0.068      -0.011       0.306
C(IJ_CODE, Treatment('ROS'))[T.AVP]         0.3564      0.069      5.195      0.000       0.222       0.491
C(IJ_CODE, Treatment('ROS'))[T.BAN]        -0.3424      0.061     -5.632      0.000      -0.461      -0.223
C(IJ_CODE, Treatment('ROS'))[T.BAR]         0.5409      0.092      5.854      0.000       0.360       0.722
C(IJ_CODE, Treatment('ROS'))[T.BAZ]        -0.4705      0.097     -4.865      0.000      -0.660      -0.281
C(IJ_CODE, Treatment('ROS'))[T.BHA]        -1.7065      0.099    -17.308      0.000      -1.900      -1.513
C

## Reading our results

That's a **lot lot lot lot of variables!** Let's convert them to an easier-to-read dataframe with odds ratios and p values.

In [25]:
import numpy as np

coefs = pd.DataFrame({
    'coef': result.params.values,
    'odds ratio': np.exp(result.params.values),
    'pvalue': result.pvalues,
    'column': result.params.index
}).sort_values(by='odds ratio', ascending=False)
coefs.head(10)

,coef,odds ratio,pvalue,column
"C(NAT, Treatment('CH'))[T.CU]",2.53550,12.62271,0.00000,"C(NAT, Treatment('CH'))[T.CU]"
"C(IJ_CODE, Treatment('ROS'))[T.TAB]",2.14198,8.51629,0.00000,"C(IJ_CODE, Treatment('ROS'))[T.TAB]"
"C(IJ_CODE, Treatment('ROS'))[T.PMM]",2.12543,8.37650,0.00000,"C(IJ_CODE, Treatment('ROS'))[T.PMM]"
"C(IJ_CODE, Treatment('ROS'))[T.ICD]",2.08921,8.07852,0.00000,"C(IJ_CODE, Treatment('ROS'))[T.ICD]"
"C(IJ_CODE, Treatment('ROS'))[T.NB]",1.78492,5.95909,0.00000,"C(IJ_CODE, Treatment('ROS'))[T.NB]"
"C(IJ_CODE, Treatment('ROS'))[T.WVW]",1.74893,5.74842,0.00000,"C(IJ_CODE, Treatment('ROS'))[T.WVW]"
"C(IJ_CODE, Treatment('ROS'))[T.OLC]",1.66564,5.28903,0.00000,"C(IJ_CODE, Treatment('ROS'))[T.OLC]"
"C(IJ_CODE, Treatment('ROS'))[T.SSC]",1.60332,4.96951,0.00000,"C(IJ_CODE, Treatment('ROS'))[T.SSC]"
"C(IJ_CODE, Treatment('ROS'))[T.PD1]",1.56840,4.79897,0.00000,"C(IJ_CODE, Treatment('ROS'))[T.PD1]"
"C(IJ_CODE, Treatment('ROS'))[T.RAC]",1.53783,4.65448,0.00000,"C(IJ_CODE, Treatment('ROS'))[T.RAC]"


While we could pick through that list to get the top `n`, let's look at each slice separately. Judges, sites, and nationalities, one at a time.

### Judges

In [26]:
# Highest odds ratio
coefs[coefs.column.str.contains("IJ_CODE")].sort_values(by='odds ratio', ascending=False).head()

,coef,odds ratio,pvalue,column
"C(IJ_CODE, Treatment('ROS'))[T.TAB]",2.14198,8.51629,0.00000,"C(IJ_CODE, Treatment('ROS'))[T.TAB]"
"C(IJ_CODE, Treatment('ROS'))[T.PMM]",2.12543,8.37650,0.00000,"C(IJ_CODE, Treatment('ROS'))[T.PMM]"
"C(IJ_CODE, Treatment('ROS'))[T.ICD]",2.08921,8.07852,0.00000,"C(IJ_CODE, Treatment('ROS'))[T.ICD]"
"C(IJ_CODE, Treatment('ROS'))[T.NB]",1.78492,5.95909,0.00000,"C(IJ_CODE, Treatment('ROS'))[T.NB]"
"C(IJ_CODE, Treatment('ROS'))[T.WVW]",1.74893,5.74842,0.00000,"C(IJ_CODE, Treatment('ROS'))[T.WVW]"


In [27]:
# Lowest odds ratio
coefs[coefs.column.str.contains("IJ_CODE")].sort_values(by='odds ratio', ascending=True).head()

,coef,odds ratio,pvalue,column
"C(IJ_CODE, Treatment('ROS'))[T.MS1]",-2.65239,0.07048,0.00000,"C(IJ_CODE, Treatment('ROS'))[T.MS1]"
"C(IJ_CODE, Treatment('ROS'))[T.HAR]",-2.64110,0.07128,0.00000,"C(IJ_CODE, Treatment('ROS'))[T.HAR]"
"C(IJ_CODE, Treatment('ROS'))[T.TCR]",-2.52388,0.08015,0.00000,"C(IJ_CODE, Treatment('ROS'))[T.TCR]"
"C(IJ_CODE, Treatment('ROS'))[T.RMH]",-2.45111,0.08620,0.00000,"C(IJ_CODE, Treatment('ROS'))[T.RMH]"
"C(IJ_CODE, Treatment('ROS'))[T.VBM]",-2.41153,0.08968,0.00000,"C(IJ_CODE, Treatment('ROS'))[T.VBM]"


Those odds ratios let us how each other judge compares to [Rafael Ortiz-Segura](https://trac.syr.edu/immigration/reports/judge2008/00170ORL/index.html), controlling for city and nationality. 


### Cities/Sites

Same with the cities - they're each compared to New York.

In [28]:
# Highest odds ratio
coefs[coefs.column.str.contains("SITE")].sort_values(by='odds ratio', ascending=False).head()

,coef,odds ratio,pvalue,column
"C(UPDATE_SITE, Treatment('NYC'))[T.ELP]",0.96261,2.61851,0.00000,"C(UPDATE_SITE, Treatment('NYC'))[T.ELP]"
"C(UPDATE_SITE, Treatment('NYC'))[T.SFR]",0.66628,1.94698,0.00000,"C(UPDATE_SITE, Treatment('NYC'))[T.SFR]"
"C(UPDATE_SITE, Treatment('NYC'))[T.PHO]",0.66577,1.94599,0.00000,"C(UPDATE_SITE, Treatment('NYC'))[T.PHO]"
"C(UPDATE_SITE, Treatment('NYC'))[T.LOS]",0.59240,1.80833,0.00000,"C(UPDATE_SITE, Treatment('NYC'))[T.LOS]"
"C(UPDATE_SITE, Treatment('NYC'))[T.WAS]",0.50693,1.66018,0.00000,"C(UPDATE_SITE, Treatment('NYC'))[T.WAS]"


In [29]:
# Highest odds ratio
coefs[coefs.column.str.contains("SITE")].sort_values(by='odds ratio', ascending=True).head()

,coef,odds ratio,pvalue,column
"C(UPDATE_SITE, Treatment('NYC'))[T.ELO]",-2.16287,0.11499,0.00000,"C(UPDATE_SITE, Treatment('NYC'))[T.ELO]"
"C(UPDATE_SITE, Treatment('NYC'))[T.LAD]",-1.67067,0.18812,0.00000,"C(UPDATE_SITE, Treatment('NYC'))[T.LAD]"
"C(UPDATE_SITE, Treatment('NYC'))[T.PIS]",-1.63704,0.19456,0.00000,"C(UPDATE_SITE, Treatment('NYC'))[T.PIS]"
"C(UPDATE_SITE, Treatment('NYC'))[T.CHL]",-1.59208,0.20350,0.00000,"C(UPDATE_SITE, Treatment('NYC'))[T.CHL]"
"C(UPDATE_SITE, Treatment('NYC'))[T.KRO]",-1.27262,0.28010,0.00000,"C(UPDATE_SITE, Treatment('NYC'))[T.KRO]"


### Nationalities

We'll need to pull in the list of nationalities so we know what we're looking at.

In [30]:
from io import StringIO

content = open("data/tblLookupAlienNat.csv").read().replace('"','')

nationalities = pd.read_csv(StringIO(content), sep='\t')
nationalities[['strCode', 'strDescription']].sort_values(by='strCode')


,strCode,strDescription
0,<A>,<All>
1,??,UNKNOWN NATIONALITY
2,AB,ARUBA
3,AC,ANTIGUA AND BARBUDA
4,AF,AFGHANISTAN
5,AG,ALGERIA
6,AL,ALBANIA
7,AM,ARMENIA
8,AN,ANDORRA
9,AO,ANGOLA


In [31]:
# Highest odds ratio
coefs[coefs.column.str.contains("NAT")].sort_values(by='odds ratio', ascending=False).head()

,coef,odds ratio,pvalue,column
"C(NAT, Treatment('CH'))[T.CU]",2.53550,12.62271,0.00000,"C(NAT, Treatment('CH'))[T.CU]"
"C(NAT, Treatment('CH'))[T.IZ]",1.38377,3.98990,0.00000,"C(NAT, Treatment('CH'))[T.IZ]"
"C(NAT, Treatment('CH'))[T.EG]",1.31471,3.72368,0.00000,"C(NAT, Treatment('CH'))[T.EG]"
"C(NAT, Treatment('CH'))[T.BZ]",1.29744,3.65991,0.00000,"C(NAT, Treatment('CH'))[T.BZ]"
"C(NAT, Treatment('CH'))[T.ER]",1.21485,3.36977,0.00000,"C(NAT, Treatment('CH'))[T.ER]"


Cuba, Iraq, Egypt, and Belarus are the top five (BZ and BS are both Belarus).

In [32]:
# Lowest odds ratio
coefs[coefs.column.str.contains("NAT")].sort_values(by='odds ratio', ascending=True).head()

,coef,odds ratio,pvalue,column
"C(NAT, Treatment('CH'))[T.HO]",-1.17124,0.30998,0.00000,"C(NAT, Treatment('CH'))[T.HO]"
"C(NAT, Treatment('CH'))[T.HA]",-0.95258,0.38575,0.00000,"C(NAT, Treatment('CH'))[T.HA]"
"C(NAT, Treatment('CH'))[T.ES]",-0.71489,0.48925,0.00000,"C(NAT, Treatment('CH'))[T.ES]"
"C(NAT, Treatment('CH'))[T.ID]",-0.70545,0.49389,0.00000,"C(NAT, Treatment('CH'))[T.ID]"
"C(NAT, Treatment('CH'))[T.EC]",-0.61495,0.54067,0.00000,"C(NAT, Treatment('CH'))[T.EC]"


Honduras, Haiti, El Salvador, Indonesia, and Ecuador are the bottom five, all of which approximately half the chance or less than an asylum-seeker from China.

## Trying again

The thing is, though, **these values probably depend on how we filter.** Let's say we filter and perform our regression again, this time the only change being **we look for nationalities and judges that show up 100 times instead of 300.**

In [33]:
common_nats_100 = list(merged.NAT.value_counts()[merged.NAT.value_counts() >= 100].index)
has_common_nat_100 = merged.NAT.isin(common_nats_100)

common_judges_100 = list(merged.IJ_CODE.value_counts()[merged.IJ_CODE.value_counts() >= 100].index)
has_common_judge_100 = merged.IJ_CODE.isin(common_judges_100)

# We'll use the same two filters for judges and sites as before
# But use our new nationalities list
filtered = merged[has_common_judge_100 & has_common_site & has_common_nat_100]

In [34]:
print("ORIGINAL NATIONALITIES")
print(common_nats)
print("")
print("ADDED NATIONALITIES")
print(list(set(common_nats_100) - set(common_nats)))

ORIGINAL NATIONALITIES
['CH', 'MX', 'ES', 'GT', 'HA', 'HO', 'CO', 'CU', 'IN', 'AL', 'ID', 'PK', 'RU', 'VE', 'ET', 'EG', 'NP', 'AM', 'PE', 'RP', 'DR', 'IR', 'CM', 'BG', 'BR', 'GV', 'NI', 'IZ', 'EC', 'NU', 'JM', 'MR', 'UR', 'SO', 'YO', 'UE', 'ER', 'KE', 'CE', 'LE', 'RO', 'GA', 'BM', 'LI', 'UZ', 'JO', 'GH', 'IV', 'ML', 'CF', 'SY', 'SL', 'KS', 'VM', 'FJ', 'TD', 'ZI', 'PL', 'TO', 'BU', 'SU', 'GO', 'CA', 'MO', 'IS', 'SG', 'UG', 'GY', 'AR', 'TU', 'MD', 'BO', 'YE', 'MG', 'SS', 'AF', 'BZ', 'KZ', 'BS', 'LA', 'UK', 'KG', 'BL', 'RW', 'MM', 'CC', 'AG', 'AZ', 'DC', 'YS', 'SF', 'CI', 'TH', 'CS', 'GE', 'TS', 'PM', 'CB', 'BY', 'PO', 'NG', 'KV', 'TZ', 'MY', 'LH', 'BH', 'CG', 'CV', 'CD']

ADDED NATIONALITIES
['CX', 'BN', 'DO', 'TN', 'BI', 'FR', 'CT', 'HU', 'HK', 'TR', 'ZA', 'GJ', 'BB', 'CR', 'IT', 'EO', 'ST', 'PN', 'GR', 'TW', 'NS', 'AO', 'LV', 'CZ', 'JA', 'SP', 'BF', 'UY', 'LY', 'PA', 'SA', 'KU', 'SR', 'AS', 'TA']


In [35]:
print(len(common_judges_100), "judges, up from", len(common_judges))

487 judges, up from 359


In [36]:
print("Old version", merged[has_common_judge & has_common_site & has_common_nationality].shape)
print("New version", filtered.shape)

Old version (548433, 14)
New version (578438, 14)


Okay, looks like an additional 30,000 cases, only about a 5% increase.

### Performing the regression

In [37]:
%%time

import statsmodels.formula.api as smf

# Create and run our regression
model = smf.logit("""
    granted ~ C(IJ_CODE, Treatment('ROS')) + C(UPDATE_SITE, Treatment('NYC')) + C(NAT, Treatment('CH'))
""", data=filtered)
result = model.fit(method='bfgs', maxiter=1000)
# Not going to print the summary because it's SO LONG
#result.summary()

Optimization terminated successfully.
         Current function value: 0.553239
         Iterations: 482
         Function evaluations: 483
         Gradient evaluations: 483
CPU times: user 16min 55s, sys: 33.7 s, total: 17min 29s
Wall time: 9min 25s


In [38]:
# Build the coefficients dataframe
coefs = pd.DataFrame({
    'coef': result.params.values,
    'odds ratio': np.exp(result.params.values),
    'pvalue': result.pvalues,
    'column': result.params.index
}).sort_values(by='odds ratio', ascending=False)

In [39]:
# Highest odds ratio
coefs[coefs.column.str.contains("NAT")].sort_values(by='odds ratio', ascending=False).head()

,coef,odds ratio,pvalue,column
"C(NAT, Treatment('CH'))[T.CU]",2.47675,11.90248,0.00000,"C(NAT, Treatment('CH'))[T.CU]"
"C(NAT, Treatment('CH'))[T.IZ]",1.36025,3.89717,0.00000,"C(NAT, Treatment('CH'))[T.IZ]"
"C(NAT, Treatment('CH'))[T.LY]",1.32737,3.77110,0.00000,"C(NAT, Treatment('CH'))[T.LY]"
"C(NAT, Treatment('CH'))[T.EG]",1.32115,3.74772,0.00000,"C(NAT, Treatment('CH'))[T.EG]"
"C(NAT, Treatment('CH'))[T.BZ]",1.25971,3.52441,0.00000,"C(NAT, Treatment('CH'))[T.BZ]"


In [40]:
# Lowest odds ratio
coefs[coefs.column.str.contains("NAT")].sort_values(by='odds ratio', ascending=True).head()

,coef,odds ratio,pvalue,column
"C(NAT, Treatment('CH'))[T.HO]",-1.19281,0.30337,0.00000,"C(NAT, Treatment('CH'))[T.HO]"
"C(NAT, Treatment('CH'))[T.HA]",-0.98694,0.37271,0.00000,"C(NAT, Treatment('CH'))[T.HA]"
"C(NAT, Treatment('CH'))[T.ES]",-0.76023,0.46756,0.00000,"C(NAT, Treatment('CH'))[T.ES]"
"C(NAT, Treatment('CH'))[T.ID]",-0.72259,0.48549,0.00000,"C(NAT, Treatment('CH'))[T.ID]"
"C(NAT, Treatment('CH'))[T.EC]",-0.65532,0.51928,0.00000,"C(NAT, Treatment('CH'))[T.EC]"


## Discussion topics

By lowering the threshold from 300 to 100, we gained over 30 "new" countries. Is there a downside to leaving more countries in there? If we think there aren't enough to make a valid conclusion, isn't that what p values are for?